In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")

Running on cuda


In [11]:
def data_loader(data, batch_size, random_seed=123, valid_size=0.1, shuffle=True,test=False):
    normalise = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),normalise])
    transform_train = transforms.Compose(
        [
        transforms.RandomRotation(30),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         normalise])
    if test:
        dataset = datasets.Flowers102(root=data, split="test", download=True, transform=transform)
        data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)
        print(f"Number of test examples: {len(dataset)}")
        return data_loader
    train_dataset = datasets.Flowers102(root=data, split="train", download=True, transform=transform_train)
    validation_dataset = datasets.Flowers102(root=data, split="val", download=True, transform=transform)
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    # train_idx, valid_idx = indices[split:], indices[:split]
    # train_sampler = torch.utils.data.sampler.SubsetRandomSampler(1020)
    # valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(1020)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=shuffle)
    valid_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=True)
    print(f"Number of training examples: {len(train_dataset)}")
    return (train_loader, valid_loader)

train_loader, valid_loader = data_loader(data='flowers', batch_size=32, shuffle=True, test=False)
test_loader = data_loader(data='flowers', batch_size=32, shuffle=True, test=True)
# normalise = transforms.Normalize(mean=[0.4914,0.4822,0.4465],std=[0.2023,0.1994,0.2010])
# transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),normalise])
# transform_train = transforms.Compose(
#     [transforms.Resize((256,256)),
#      transforms.RandomHorizontalFlip(),
#      transforms.RandomVerticalFlip(p=0.5),
#      transforms.CenterCrop(224),
#      transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.4, hue=0.1),
#      transforms.RandomRotation(10),
# #      transforms.ToTensor(),
# #      normalise])
# train_dataset = datasets.Flowers102(root='./data', split="train", download=True, transform=transform_train)
# validation_dataset = datasets.Flowers102(root="./data/", split="val", download=True, transform=transform)
# test = datasets.Flowers102(root="./data", split="test", download=True, transform=transform)
# train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
# valid_loader = DataLoader(dataset=validation_dataset, batch_size=32, shuffle=False)
# test_loader = DataLoader(dataset=test, batch_size=32, shuffle=False)


Number of training examples: 1020
Number of test examples: 6149


In [12]:
class Bnb(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bnb, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)

        self.relu = nn.ReLU()
        self.i_downsample = i_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.conv3(x)
        x = self.bn3(x)

        if self.i_downsample is not None:
            identity = self.i_downsample(identity)

        x += identity
        x = self.relu(x)

        return x

In [13]:
class Rn(nn.Module):
    def __init__(self, block, layers, num_classes=102, num_channels=3):
        super(Rn, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, layers[0], planes=64)
        self.layer2 = self._make_layer(block, layers[1], planes=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], planes=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], planes=512, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(512*block.expansion, num_classes)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)


        x = self.avg_pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x
    
    def _make_layer(self, block, num_residual_blocks, planes, stride=1):
        ii_downsample = None
        layers = []

        if stride != 1 or self.in_channels != planes*block.expansion:
            ii_downsample = nn.Sequential(nn.Conv2d(self.in_channels, planes*block.expansion, kernel_size=1, stride=stride),
                                          nn.BatchNorm2d(planes*block.expansion))
        layers.append(block(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*block.expansion

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, planes))
        
        return nn.Sequential(*layers)
    

In [14]:
num_classes = 102
num_epochs = 300
learning_rate = 0.0001
weight_decay = 1e-4
print_every = 5
model = Rn(Bnb, [3, 4, 6, 3], num_classes=102).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)
print(train_loader)
total_step = len(train_loader)

In [15]:
def count_parameters(model)->int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

23743526


In [16]:
import requests
def post_discord(epoch,train_loss, val_accuracy):
    data = {
        "username": "Flower Classifier - BB"
    }
    data["embeds"] = [
        {
            "title": "Epoch {} Results".format(epoch),
            "color": 0x00ff00,
            "fields": [
                {
                    "name": "Training Loss",
                    "value": f"{train_loss}",
                    "inline": False
                },
                {
                    "name": "Validation Accuracy",
                    "value": f"{val_accuracy}",
                    "inline": False
                },
            ]
        }
    ]
    r = requests.post("https://discord.com/api/webhooks/1052587856343867392/SnR2U4HIdeF6ShECr0k1Mit6yrl3HgjtCMk_LykGD8eQ1qsZViY8HLeYsoBUXOHYfbgP", json=data)
    print(r.status_code)


In [17]:
import gc

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        gc.collect()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    
    if epoch % print_every == 0:
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            correct = 0
            total = 0
            correctT = 0
            totalT = 0
            for images, labels in valid_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted==labels).sum().item()
            for images, labels in train_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                totalT += labels.size(0)
                correctT += (predicted==labels).sum().item()

            print('Training Accuracy accross {} images: {} %'.format(totalT, 100 * correctT / totalT))
            print('Validation Accuracy accross {} images: {} %'.format(total, 100 * correct / total))
            post_discord(epoch+1, '{:.4f}'.format(loss.item()), 100 * correct / total)
    if epoch % 20 == 0 && epoch != 0:
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                del images, labels, outputs
        torch.save(model.state_dict(), 'model-{}.pth'.format(100 * correct / total))


Epoch [1/300], Loss: 4.6353
Training Accuracy accross 1020 images: 1.4705882352941178 %
Validation Accuracy accross 1020 images: 1.1764705882352942 %
204
Epoch [2/300], Loss: 4.8559
Epoch [3/300], Loss: 4.6737
Epoch [4/300], Loss: 4.6871
Epoch [5/300], Loss: 4.5588
Epoch [6/300], Loss: 4.5664
Training Accuracy accross 1020 images: 2.3529411764705883 %
Validation Accuracy accross 1020 images: 2.7450980392156863 %
204
Epoch [7/300], Loss: 4.4934
Epoch [8/300], Loss: 4.4075
Epoch [9/300], Loss: 4.3427
Epoch [10/300], Loss: 4.4343
Epoch [11/300], Loss: 4.5419
Training Accuracy accross 1020 images: 4.509803921568627 %
Validation Accuracy accross 1020 images: 4.313725490196078 %
204
Epoch [12/300], Loss: 4.3576
Epoch [13/300], Loss: 4.4437
Epoch [14/300], Loss: 4.3936
Epoch [15/300], Loss: 4.1972
Epoch [16/300], Loss: 4.2379
Training Accuracy accross 1020 images: 7.352941176470588 %
Validation Accuracy accross 1020 images: 5.980392156862745 %
204
Epoch [17/300], Loss: 4.2651
Epoch [18/300], 

In [ ]:
torch.save({
    "epoch": epoch,
    "model_state_dict": model.state_dict(), 
    "optimizer_state_dict": optimizer.state_dict(),
    "loss": loss,
    'checkpoints/model_resnet50_300_v1.ckpt'
})

In [ ]:
torch.save(model.state_dict(), 'model_5.pth')

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))   


In [ ]:
def count_parameters(model)->int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

In [ ]:
print(model)